**Importando bibliotecas que serão úteis para o Catboost classifier e determinando uma random_seed**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.preprocessing import LabelEncoder

#Set seed
seed_ = 194
np.random.seed(seed_)

**Carregando a base de dados a ser utilizada**

In [ ]:

df = pd.read_csv('datasets/df_sem_duplicatas.csv')

**Set-Up do LabelEncoder model**

In [ ]:
#Criação do LabelEncoder
pkl_file = open('models/label_encoder.pkl', 'rb')
le = pickle.load(pkl_file) 
pkl_file.close()

#O código abaixo é para aplicar na coluna categoria o LabelEncoder
le.fit(df['categoria'])
df['categoria'] = le.transform(df['categoria'])

#Atribuição das classes do LabelEncoder a variável target_names
target_names = le.classes_

**Separando em treino e teste**

In [ ]:
# Separando em variaveis independentes e dependentes
X, y = df.drop('categoria', axis = 1), df['categoria']

# Separando em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed_)
data_train = pd.concat([X_train, y_train], axis = 1)

# Separando em teste e validação
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = seed_)

# Gerando TF-IDF
TfidfVec = TfidfVectorizer(ngram_range = (1, 1))
X_train_tfidf = TfidfVec.fit_transform(X_train['descricao'].tolist())
X_val_tfidf = TfidfVec.transform(X_val['descricao'].tolist())
X_test_tfidf = TfidfVec.transform(X_test['descricao'].tolist())

# Salvando tf-idf como pickle
with open('models/tfidf_catboost.pkl', 'wb') as model_file:
    pickle.dump(TfidfVec, model_file)

**Set up Modelo Castboost**

Impressão de metricas e treinamento com o GridSearchCV

In [ ]:
#a função report_modelo é para imprimir as métricas do modelo
def report_modelo(modelo, y, pred):
    print("Acurácia: %f" % accuracy_score(y_true = y, y_pred = pred))
    
    # Plotando a matriz de confusão
    cm = confusion_matrix(y, pred)
    cm = pd.DataFrame(cm, index = target_names, columns = target_names)

    plt.figure(figsize = (8, 8))
    sns.heatmap(cm, cmap = "Blues", linecolor = 'black', linewidths = 1, annot = True, \
                fmt = '', xticklabels = target_names, yticklabels = target_names)
    plt.show()
    
    # Imprimindo métricas
    print(classification_report(y_true = y, y_pred = pred, target_names= target_names))

In [ ]:
#A função treinaClassificacao_GridSearchCV é para treinar o modelo com o GridSearchCV
def treinaClassificacao_GridSearchCV(modelo, params_, x_treino, y_treino, x_teste, y_teste,\
                        n_jobs = -1, cv = 5, refit = True, scoring = None, salvar_resultados = False,\
                       report_treino = False, retorna_modelo = False):
    
    # Realizando treinamento baseado no grid informado
    grid = GridSearchCV(modelo, params_, n_jobs = n_jobs, cv = cv, refit = refit, scoring = scoring)
    
    grid.fit(x_treino, y_treino)
    pred = grid.predict(x_teste)
    modelo_ = grid.best_estimator_

    print(grid.best_params_)
    
    # Realizando previsões
    y_test_pred = modelo_.predict(x_teste)
    
    # Gerando report do modelo
    report_modelo(modelo_, y_teste, y_test_pred) 
    
    # Salvando resultados em dataframe caso solicitado
    if salvar_resultados:
        resultados_df = pd.DataFrame(grid.cv_results_)
        
        if retorna_modelo:
            return resultados_df, modelo_
        else:
            resultados_df
        
    if retorna_modelo:
        return modelo_

**Treinando com o CatBoostClassifier**

In [ ]:
#o codigo abaixo é para treinar o modelo CatBoostClassifier
%%time

params = {}

# Modelo Base CatBoost Classifier
modelo_cat = CatBoostClassifier(iterations = 50, eval_metric = 'MultiClass', metric_period = 50,\
                                learning_rate = 0.01, verbose = False, random_seed = seed_)

treinaClassificacao_GridSearchCV(modelo_cat, params, X_train_tfidf, y_train, X_test_tfidf, y_test)

**Tunning do Catboost**

In [ ]:
%%time

params = {
    'iterations': [50, 100, 500, 1000],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Tuning 01 Catboost
modelo_cat = CatBoostClassifier(eval_metric = 'MultiClass', metric_period = 50,\
                                verbose = False, random_seed = seed_)

treinaClassificacao_GridSearchCV(modelo_cat, params, X_train_tfidf, y_train, X_test_tfidf, y_test)

In [ ]:
%%time

params = {
    'iterations': [750, 1000, 1250],
    'learning_rate': [0.2, 0.1, 0.01]
}

# Tuning 02 Catboost
modelo_cat = CatBoostClassifier(eval_metric = 'MultiClass', metric_period = 50,\
                                verbose = False, random_seed = seed_)

treinaClassificacao_GridSearchCV(modelo_cat, params, X_train_tfidf, y_train, X_test_tfidf, y_test)

In [ ]:
%%time

params = {
    'iterations': [1250, 1500, 2000],
    'learning_rate': [0,4, 0.3, 0.2]
}

# Tuning 03 Catboost
modelo_cat = CatBoostClassifier(eval_metric = 'MultiClass', metric_period = 50,\
                                verbose = False, random_seed = seed_)

treinaClassificacao_GridSearchCV(modelo_cat, params, X_train_tfidf, y_train, X_test_tfidf, y_test)

In [ ]:
%%time

params = {
    'iterations': [1250],
    'learning_rate': [0.2],
    'depth': [6, 7, 8]
}

# Tuning 04 Catboost
modelo_cat = CatBoostClassifier(eval_metric = 'MultiClass', metric_period = 50,\
                                verbose = False, random_seed = seed_)

treinaClassificacao_GridSearchCV(modelo_cat, params, X_train_tfidf, y_train, X_test_tfidf, y_test)

**Salvando o Modelo**

In [ ]:
%%time

params = {}

# Tuning 03 Catboost
modelo_cat = CatBoostClassifier(iterations = 1250, learning_rate = 0.2, eval_metric = 'MultiClass',\
                                metric_period = 50, verbose = False, random_seed = seed_)

modelo_cat = treinaClassificacao_GridSearchCV(modelo_cat, params, X_train_tfidf, y_train,\
                                              X_test_tfidf, y_test, retorna_modelo = True)

In [2]:
modelo_cat.save_model('models/catboost.model')

#Salvando modelo como pickle
with open('models/catboost.pkl', 'wb') as model_file:
    pickle.dump(modelo_cat, model_file)

NameError: name 'modelo_cat' is not defined